### 실습 전 설치해야할 라이브러리 및 명령어
```!pip install newspaper3k==0.2.8```<br>
```!pip install lxml-html-clean==0.1.1```

---
### 9.4.1 뉴스 기사 수집하기

In [11]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article

code = requests.get('https://www.joongang.co.kr/money')
html = BeautifulSoup(code.text, 'html.parser')
title_list = html.select('ul.card_right_list.rank_list h2.headline > a')

# 기사 제목 띄우기
print('|| --- [중앙일보 경제 뉴스]')

for index, title in enumerate(title_list):
    print(f'|| --- {index + 1} - {title.text.strip()}')

print('||')

# 수집할 기사 선택
user_choice = int(input('|| --- 확인할 뉴스를 선택하세요 :'))

# 선택한 기사의 본문 링크 가져오기
user_item = title_list[user_choice - 1]
news_title = user_item.text
news_link = user_item.attrs['href']

# newspaper 모듈로 기사 본문 가공 및 출력
article = Article(news_link, language='ko')
article.download()
article.parse()
news_content = article.text

print('└└\n', news_content)

|| --- [중앙일보 경제 뉴스]
|| --- 1 - 5억에 사놓고 200억에 판다…재벌 2세의 옥수동 알박기?
|| --- 2 - 윤 대통령, 7일 대국민담화·기자회견…명태균·김여사 설명한다
|| --- 3 - 백종원보다 입맛 좋았다…'새로' 대박낸 전자 혀의 비밀
|| --- 4 - '모다모다 샴푸' 망한거 아냐? 100억 사옥 팔고 돌아왔다
|| --- 5 - 공항에 뜬 거대한 은색 차…지드래곤, 국내 미출시 테슬라 탔다
||


|| --- 확인할 뉴스를 선택하세요 : 1


└└
 ‘알박기’란 개발사업이 추진되고 있는 곳의 일부 땅(사업에 꼭 필요한 땅)을 매입한 후 개발사업자에게 고가로 되파는 부동산 투기수법을 말합니다. 알박기는 사업을 지연시키고, 필요 이상으로 사업비를 증대시키기 때문에 부동산 투기수법 중에서도 ‘악질’로 꼽힙니다.

그런데 요즘 서울 한복판의 한 재건축 사업장이 ‘알박기 논란’으로 시끄럽습니다. 주민들에 따르면 이 아파트의 유일한 차량 통행로(주 출입로)의 일부(약 40평) 등을 5억여원에 산 사람이 주민들에게 그 땅을 200억원대에 매입하라고 한다는 것입니다.

특히 그 ‘땅 주인’이 30대 그룹 창업자의 외아들이어서 더 논란이 큽니다.

논란이 일고 있는 아파트는 서울 성동구 옥수동의 한남 하이츠(28평~55평, 535가구)입니다. 지은 지 40년(1982년 준공)이 넘은 단지로 현재 재건축을 추진 중인데, 2020년 정비사업 사업시행인가(조건부)를 받은 후 아직 다음 진도를 못 나가고 있습니다.

40평 땅에 발목 잡힌 500여 가구 재건축

진도를 못 나간 여러 가지 요인 중 하나가 ‘입구 땅 문제’입니다. 입구 땅 문제란 이 아파트 주민들이 40여 년간 주 진입로로 쓰고 있는 도로 중 일부가 몇 년 전 ‘개인 땅’으로 변경됐는데, 재건축 사업의 인허가권을 쥔 성동구청에서는 재건축 공사 시작 전까지 ‘이 문제에 대한 조치 계획을 제출할 것’을 사업시행인가 ‘조건’으로 내걸었습니다.

한 주민은 “당연히 단지 주민들의 공동 소유지라고 생각했던 주 출입로의 일부가 개인 땅이라는 것도 놀랍지만, 재건축 사업을 위해 터무니없이 비싼 가격으로 그 땅을 사는 방안이 논의되고 있다는 게 믿겨지지 않는다”고 말했습니다.

주 출입로의 일부가 현재의 땅 주인에게 넘어간 과정은 이렇습니다. 주 출입로 중 일부 땅을 포함한 4개 필지(총 220여 평)가 이 아파트를 지은 중앙산업 소유로 남아 있었는데, 중앙산업이 도산하면서 땅 소유주는 국제자산신탁으로 바뀝니다.

이후 한 개인이 2018년 6월 이 4필지의 새로운 땅 

---
### 9.4.2 문서 요약 API 키 발급받기

In [14]:
# 리파지토리에 올리기 전에 반드시 빈 값으로 대체하기.
client_id = ''
client_secret = ''

---
### 9.4.3 뉴스 기사 요약하기

In [15]:
# news_content의 글자 수를 1900자로 자르기
if len(news_content) >= 1900:
    news_content = news_content[:1900]

import json

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}

data = {
    "document": {
        "title": news_title,
        "content": news_content
    },
    "option": {
        "language": "ko",
        "model": "news",
        "tone": 3,
        "summaryCount": 6
    }
}

url = f"https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize"

response = requests.post(url, data=json.dumps(data), headers=headers)
response_dict = json.loads(response.text)

print('[문서 요약 결과]')
print(response_dict['summary'])

[문서 요약 결과]
'알박기'란 개발사업이 추진되고 있는 곳의 일부 땅(사업에 꼭 필요한 땅)을 매입한 후 개발사업자에게 고가로 되파는 부동산 투기수법을 말함.
입구 땅 문제란 이 아파트 주민들이 40여 년간 주 진입로로 쓰고 있는 도로 중 일부가 몇 년 전 '개인 땅'으로 변경됐는데, 재건축 사업의 인허가권을 쥔 성동구청에서는 재건축 공사 시작 전까지 '이 문제에 대한 조치 계획을 제출할 것'을 사업시행인가 '조건'으로 내걺.
한 주민은 "당연히 단지 주민들의 공동 소유지라고 생각했던 주 출입로의 일부가 개인 땅이라는 것도 놀랍지만, 재건축 사업을 위해 터무니없이 비싼 가격으로 그 땅을 사는 방안이 논의되고 있다는 게 믿겨지지 않는다"고 말함.
주 출입로의 일부가 현재의 땅 주인에게 넘어간 과정은 이럼.
주 출입로 중 일부 땅을 포함한 4개 필지(총 220여 평)가 이 아파트를 지은 중앙산업 소유로 남아 있었는데, 중앙산업이 도산하면서 땅 소유주는 국제자산신탁으로 바뀜.
이후 한 개인이 2018년 6월 이 4필지의 새로운 땅 주인이 되는데, 이 아파트 재건축조합 설립인가(2018년 5월)가 난 직후.


---
### 9.4.4 요약 결과를 음성으로 변환하기

### 실습 전 설치해야할 라이브러리 및 명령어
```!pip install gtts==2.5.1```<br>
```!pip install playsound==1.2.2```

In [19]:
from gtts import gTTS
from playsound import playsound

comment_to_voice = gTTS(text=response_dict['summary'], lang='ko')
comment_to_voice.save('news.mp3')
playsound('news.mp3')

---
### 최종코드

In [22]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import json

from gtts import gTTS
from playsound import playsound

code = requests.get('https://www.joongang.co.kr/money')
html = BeautifulSoup(code.text, 'html.parser')
title_list = html.select('ul.card_right_list.rank_list h2.headline > a')

# 기사 제목 띄우기
print('|| --- [중앙일보 경제 뉴스]')

for index, title in enumerate(title_list):
    print(f'|| --- {index + 1} - {title.text.strip()}')

print('||')

# 수집할 기사 선택
user_choice = int(input('|| --- 확인할 뉴스를 선택하세요 :'))

# 선택한 기사의 본문 링크 가져오기
user_item = title_list[user_choice - 1]
news_title = user_item.text
news_link = user_item.attrs['href']

# newspaper 모듈로 기사 본문 가공 및 출력
article = Article(news_link, language='ko')
article.download()
article.parse()
news_content = article.text

print('└└\n', news_content)

# 리파지토리에 올리기 전에 반드시 빈 값으로 대체하기.
client_id = ''
client_secret = ''

# news_content의 글자 수를 1900자로 자르기
if len(news_content) >= 1900:
    news_content = news_content[:1900]

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}

data = {
    "document": {
        "title": news_title,
        "content": news_content
    },
    "option": {
        "language": "ko",
        "model": "news",
        "tone": 3,
        "summaryCount": 6
    }
}

url = f"https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize"

response = requests.post(url, data=json.dumps(data), headers=headers)
response_dict = json.loads(response.text)

print('\n[문서 요약 결과]')
print(response_dict['summary'])

comment_to_voice = gTTS(text=response_dict['summary'], lang='ko')
comment_to_voice.save('news.mp3')
playsound('news.mp3')

|| --- [중앙일보 경제 뉴스]
|| --- 1 - 5억에 사놓고 200억에 판다…재벌 2세의 옥수동 알박기?
|| --- 2 - 윤 대통령, 7일 대국민담화·기자회견…명태균·김여사 설명한다
|| --- 3 - 백종원보다 입맛 좋았다…'새로' 대박낸 전자 혀의 비밀
|| --- 4 - '모다모다 샴푸' 망한거 아냐? 100억 사옥 팔고 돌아왔다
|| --- 5 - 공항에 뜬 거대한 은색 차…지드래곤, 국내 미출시 테슬라 탔다
||


|| --- 확인할 뉴스를 선택하세요 : 5


└└
 가수 지드래곤(36·본명 권지용)이 미국 테슬라의 전기 픽업트럭인 '사이버트럭'을 타고 이동하는 모습이 4일 포착됐다.

지드래곤은 이날 오후 샤넬 '2024/25 크루즈 레플리카 쇼' 참석을 위해 인천국제공항을 통해 홍콩으로 출국했다. 지드래곤은 공항으로 오면서 국내 미출시 모델인 사이버트럭을 타고 와 언론의 카메라 플래시 세례를 받았다.

사이버트럭은 지난해 11월 테슬라가 출시한 전기 픽업트럭으로, 방탄 기능과 함께 기하학적인 디자인으로 출시 당시 세계적인 주목을 받았다. 당시 테슬라는 사이버트럭이 스테인리스강 합금 소재로 만들어 총알도 견딜 수 있을 정도로 단단하다고 밝혔다. 또 5톤(t) 이상의 견인력을 갖고 있고, 포르쉐를 견인하면서도 포르쉐를 앞지를 수 있다고 홍보했다.

지난달 23일(현지시간) 테슬라가 발표한 실적보고서에 따르면 테슬라는 "사이버트럭이 3분기 미국에서 모델 Y와 모델 3에 이어 세 번째로 많이 팔린 전기차가 됐다"고 밝혔다. 그러면서도 사이버트럭의 판매 수치는 공개하지 않았다.

사이버트럭은 사전 예약 물량이 많아 한국의 출시 예정일은 아직 발표되지 않았다. 자동차 업계 등에 따르면 사이버트럭 가격은 6만990달러(약 8356만원)에서 9만9990달러(약 1억3696만원)로 형성돼있다.

사이버트럭이 국내에서 판매되지 않아 지드래곤이 이날 타고 온 차량엔 임시 번호판이 붙어 있었다. 해당 차량이 홍보 등 다른 목적이 있는지나 지드래곤의 개인 소유인지는 알려지지 않았다.

지드래곤은 지난달 30일 방영된 tvN '유 퀴즈 온 더 블럭'에 출연했을 땐 녹화장으로 오면서 가격이 수억 원대로 알려진 롤스로이스 팬텀을 타고 나타나 이목을 끌었다.

한편 지드래곤은 지난달 31일 신곡 '파워'(POWER)를 발매했다. 이는 2017년 미니 음반 '권지용' 이후 7년 4개월 만에 내놓은 그의 솔로곡이다.

[문서 요약 결과]
가수 지드래곤(36·본명 권지용)이 미국 테슬라의 전기 픽업트럭인 '사이버트럭'을 타고 이동하는 모습이 4일 